In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from netgen.csg import *
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.comp import ConvertOperator

import time

$$
\DeclareMathOperator{\Grad}{grad}
\DeclareMathOperator{\Curl}{curl}
\DeclareMathOperator{\Div}{div}
\DeclareMathOperator{\R}{\mathbb{R}}
\DeclareMathOperator{\real}{real}
$$
### Harmonische Magnetisierung

Finde $ u \in H(\Curl)$ sodass 
$$
\int_{\Omega} \mu^{-1} \Curl u \Curl v + i\omega\sigma \int_{\Omega\setminus\text{magnet}}  u  v  + 10^{-8} \int_{\text{magnet}} u v = \int_{\Omega} M \Curl v \qquad  \forall \, v \in H(\Curl) \tag{2}
$$

mit den Werten
* magnet: $\mu_r = 1, \sigma = 0$ 
* copper plate: $\mu_r = 1-6.4\cdot10^{-6}, \sigma = 58 \cdot 10^8$
* air: $\mu_r = 1, \sigma = 10^{-5}$ 

In [ ]:
geo = CSGeometry()
box_size = 4
box = OrthoBrick(Pnt(-box_size,-box_size,-box_size), Pnt(box_size,box_size,box_size)).bc("outer")
magnet = Cylinder(Pnt(0.05,0,0), Pnt(0.05,0,1), 0.3) * \
            OrthoBrick (Pnt(-1,-1,-1),Pnt(1,1,1)).mat("magnet").maxh(0.2)

plate = OrthoBrick (Pnt(-1, 1,-1.5),Pnt(1,1.2,1.5)).mat('copper').maxh(0.1)

air = (box - magnet- plate).mat('air').maxh(0.5)

geo.Add(plate)#, col=(0,1,0)) # what is col?
geo.Add(magnet)
geo.Add(air, transparent=True)

mesh = Mesh(geo.GenerateMesh(maxh=0.5))
mesh.Curve(3)


from math import pi
mu0 = 4*pi*1e-7
mur = CoefficientFunction( [1 if mat== "magnet" else 1-6.4*1e-7 if mat=='copper' else  1
                            for mat in mesh.GetMaterials()]) 

sigma = CoefficientFunction( [0 if mat== "magnet" else 58*1e8 if mat=='copper' else 1e-5
                            for mat in mesh.GetMaterials()]) 
omega = Parameter(100)

mag = CoefficientFunction((0,0,1)) * \
        CoefficientFunction( [100 if mat == "magnet" else 0 for mat in mesh.GetMaterials()])

kappa = CoefficientFunction( [1e-8 if mat== "magnet" else 1j*omega*sigma for mat in mesh.GetMaterials()])
nu = 1/(mu0*mur)

TOL = 1e-8
                            

def show (t=1):
    Draw ((-1j*omega*exp(1j*omega*t)*gfu).real, mesh, "E-field", draw_surf=False)
    Draw (sigma*(-1j*omega*exp(1j*omega*t)*gfu).real, mesh, "j", draw_surf=False)
    ## to see: clipping (0,1,0,-1)

### Ohne Preconditioner mit CG-Verfahren

In [ ]:
# with TaskManager(): # pajetrace=10**8
    
#     fes = HCurl(mesh, order = 0, dirichlet="outer", nograds=True, complex= True)
#     u,v = fes.TnT()
    
#     a = BilinearForm(fes)
#     a += nu*curl(u)*curl(v)*dx + kappa*u*v*dx
#     a.Assemble()
    
#     f = LinearForm(fes)
#     f += mag*curl(v)*dx(mesh.Materials("magnet"))
#     f.Assemble()

#     gfu = GridFunction(fes)

#     help(solvers.CG)
#     solvers.CG(sol=gfu.vec, rhs=f.vec, mat=a.mat) 

'''Fehlermeldung: either pre or freedofs must be given'''

### Mit Preconditioner 'BDDC' und CG-Verfahren

In [ ]:
start = time.time()
with TaskManager(): # pajetrace=10**8

    fes = HCurl(mesh, order = 0, dirichlet="outer",  complex= True)
    u,v = fes.TnT()

    a = BilinearForm(fes)
    a += nu*curl(u)*curl(v)*dx + kappa*u*v*dx
    
    c = Preconditioner(a, "bddc")

    f = LinearForm(fes)
    f += mag*curl(v)*dx (mesh.Materials("magnet"))
    
    a.Assemble() # don't call before preconditioner!

    f.Assemble()

    gfu = GridFunction(fes)

#     help(solvers.CG)
    solvers.CG(sol=gfu.vec, rhs=f.vec, mat=a.mat, pre=c.mat, tol = TOL) 
    stop = time.time()
    
print("calculation took {} seconds".format(stop-start))
show()

### Mit Preconditioner aus 'ams.py' und CG-Verfahren

In [ ]:
# nu = 1
# kappa = 0.001
COMPLEX = True
# TOL = 1e-5

start = time.time()
with TaskManager(): # pajetrace=10**8

    fes = HCurl(mesh, order = 0, dirichlet="outer", complex= COMPLEX)
    u,v = fes.TnT()
    
    a = BilinearForm(fes)
    a += nu*curl(u)*curl(v)*dx + kappa*u*v*dx
    a.Assemble()


    fespot = H1(mesh, order=1, complex= COMPLEX)
    upot, vpot = fespot.TnT()
    apot = BilinearForm(kappa*grad(upot)*grad(vpot)*dx + kappa*upot*vpot*dx)
    cpot = Preconditioner(apot, "h1amg")
    
#     help(cpot)
    apot.Assemble()
    
    embedpot = ConvertOperator(fespot, fes, grad(upot))
    
    fesvec = VectorH1(mesh, order=1, complex= COMPLEX, dirichlet="outer")
    uvec, vvec = fesvec.TnT()
    avec = BilinearForm(nu*InnerProduct(Grad(uvec), Grad(vvec))*dx + kappa*uvec*vvec*dx).Assemble()
    embedvec = ConvertOperator(fesvec, fes)
    
    jacobi = a.mat.CreateSmoother(fes.FreeDofs())
    
#     help(embedpot)
    
#     print(type(embedpot), type(cpot.mat), type(embedpot.CreateTranspose()))
    
    pre = embedpot @ cpot.mat @ embedpot.CreateTranspose() + \
        embedvec @ avec.mat.Inverse(inverse="sparsecholesky") @ embedvec.CreateTranspose() +\
        jacobi
    
    f = LinearForm(fes)
    f += mag*curl(v)*dx(mesh.Materials("magnet"))
    f.Assemble()
    

    
    gfu = GridFunction(fes)
    
    print('solve')
    solvers.CG(sol=gfu.vec, rhs=f.vec, mat=a.mat, pre=pre, tol = TOL) 
    stop = time.time()
    
print("calculation took {} seconds".format(stop-start))
    
show()

### Mit Preconditioner aus 'ams2.py' und CG-Verfahren

In [ ]:
COMPLEX = True
TOL = 1e-12

start = time.time()
with TaskManager(): # pajetrace=10**8

    fes = HCurl(mesh, order = 0, dirichlet="outer",  complex= COMPLEX)
    u,v = fes.TnT()
    
    a = BilinearForm(fes)
    a += nu*curl(u)*curl(v)*dx + kappa*u*v*dx
    a.Assemble()


    fespot = H1(mesh, order=1, complex= COMPLEX)
    upot, vpot = fespot.TnT()
    apot = BilinearForm(kappa*grad(upot)*grad(vpot)*dx + kappa*upot*vpot*dx)
    cpot = Preconditioner(apot, "h1amg")
    apot.Assemble()
    embedpot = ConvertOperator(fespot, fes, grad(upot))
    
    ## use vector H1 = (H1xH1xH1)
    fesvec = VectorH1(mesh, order=1, complex= COMPLEX, dirichlet="outer")
    fesscal = H1(mesh, order=1, complex= COMPLEX, dirichlet="outer")
    uscal, vscal = fesscal.TnT()
    ascal = BilinearForm(nu*grad(uscal)*grad(vscal)*dx + kappa*uscal*vscal*dx)
    cscal = Preconditioner(ascal, "h1amg")
    ascal.Assemble()
    
    cvec = 0.0*IdentityMatrix(fesvec.ndof, complex = COMPLEX) \
        + sum( [ei@cscal@ei.T for ei in (fesvec.Embedding(j) for j in range(3))] )
    
#     cvec = 0.0*IdentityMatrix(fesvec.ndof, complex = COMPLEX)
#     for j in range(3):
#         embi = Embedding(fesvec.ndof, fesvec.Range(j), complex = COMPLEX)
#         cvec = cvec + embi @ cscal @ embi.T

    embedvec = ConvertOperator(fesvec, fes)
    
    jacobi = a.mat.CreateSmoother(fes.FreeDofs())
    
    
    pre = embedpot @ cpot.mat @ embedpot.CreateTranspose() \
        + embedvec @ cvec @ embedvec.CreateTranspose() + jacobi
    
    f = LinearForm(fes)
    f += mag*curl(v)*dx(mesh.Materials("magnet"))
    f.Assemble()
    
    gfu = GridFunction(fes)
    
    print('solve')
    solvers.CG(sol=gfu.vec, rhs=f.vec, mat=a.mat, pre=pre, tol = TOL) 
    stop = time.time()
    
print("calculation took {} seconds".format(stop-start))
    
show()

$$
\DeclareMathOperator{\Grad}{grad}
\DeclareMathOperator{\Curl}{curl}
\DeclareMathOperator{\Div}{div}
\DeclareMathOperator{\R}{\mathbb{R}}
\DeclareMathOperator{\real}{real}
$$
## Notizen für mich

Fragen/Bemerkungen:
* wird im argument "pre" $C$ oder $C^{-1}$ übergeben (ich glaube letzteres)?


nützliche links:

* https://www.researchgate.net/publication/225410102_Auxiliary_space_preconditioning_in_H0curl_O
* http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.581.5885&rep=rep1&type=pdf (Lemma 2.2)
* http://www.numa.uni-linz.ac.at/Teaching/PhD/Finished/zaglmayr (ab S115)
* https://www.asc.tuwien.ac.at/~schoeberl/wiki/index.php/Numpde20
* https://nemesis.asc.tuwien.ac.at/index.php/s/HgeCgy5Xbgo4NW6/authenticate/showShare
* https://www.asc.tuwien.ac.at/~schoeberl/wiki/lva/notes/maxwell.pdf
* https://www.math-linux.com/mathematics/linear-systems/article/preconditioned-conjugate-gradient-method

ToDos:

- [ ] Theorie zu H1 AMG
- [ ] Vergleich Preconditioner (Zeit)
- [ ] Slider über die Zeit
- [ ] Spule statt Permanentmagnet
- [ ] siehe Kommentare in Markdown Zellen 

### Basics

Berachtete Beispiele iterativer Verfahren: 

* Richardson Iteration
* CG-Verfahren

Betrachtete Preconditioner:

* H1 AMG (im H1)
* Additive Schwarz (für Zusammensetzung)
* Jacobi


#### Was sind Preconditioner?

Für eine positiv definite symmetrische Matrix $A \in \R^{n\times n} $ soll das Problem 
$$
Au = f \tag{1}
$$
iterativ gelöst werden (Fixpunktiteration). Ein Precomditioner $C$ dient der Verbesserung der Konvergenzrate und sollte erfüllen:

1. $C^{-1} v$ ist für $v \in \R^n$ leicht zu berechnen
2. $C$ ist eine gute Approximation für $A$ im Sinne, dass
$$
\exists \gamma_1, \gamma_2  > 0 : ~ \gamma_1 \leq \frac{u^TAu}{u^TCu} \leq  \gamma_2 ~\forall 0  \neq u \in \R^n 
$$
und $\kappa := \frac{\gamma_2}{\gamma_1}$ möglichst nahe bei 1 (spektrale Konditionszahl von $C^{-1}A$). 

<!-- _________

Randnotiz: warum ist $\kappa$ spektrale Konditionszahl von $C^{-1}A$? \
Wiederholung: Für eine symmetrische positiv definite Matrix $M$, die euklidische Norm $||~.||$ und die Abbildung $f: b \mapsto M^{-1}b = : x$ gilt 
$$\text{cond}(M) := \text{cond} (f) = \lim \sup_{b \in \R^n} \frac{\frac{||\Delta(M^{-1}b)||}{||M^{-1}b}||}{\frac{||\Delta b||}{||b||}} = \lim \sup_{b \in \R^n} \left(\frac{||A^{-1}(\Delta b)||}{||\Delta b||} \frac{||b||}{||A^{-1}b||}\right) = \lim \sup_{b \in \R^n} \frac{||A^{-1}(\Delta b)||}{||\Delta b||} \lim \sup_{x \in \R^n} \frac{||Ax||}{||x||} = \frac{\lambda_{\max}}{\lambda_{\min}}$$


Fertig machen wenn Zeit
_________
 -->
 

#### Richardson Iteration

Wdh Numerik: Erweiterung von (1) mit $Cu$, wobei $C$ regulär ist, gibt

$$
(1) ~ \Leftrightarrow ~ Cu = Cu + f - Au  ~ \Leftrightarrow ~ u = u + C^{-1}(f-Au).
$$

Nach Einführung eines zusätzlichen Faktors $\tau$ (wozu ist das?) definiere die Fixpunktiteration 

$$
u^{k+1} := u^k + \tau C^{-1}(f-Au)
$$

deren Grenzwert die Lösung von (1) ist (bis auf das $\tau$).


#### CG-Verfahren
Wdh Numerik: Wähle Suchrichtungen $d^k$ paarweise orthogonal bzgl des Energie-Skalarprodukts.

1. Startwerte: Startpunkt $u^0$, Residuum, $r^0 = b- Au^0$, Suchrichtung $d^0 = r^0$
2. wiederhole für $k = 0,.., \min \{ $ gewünschte Genauigkeit erreicht, $ n  \}$: \
$ u^{k+1} = u^k + \alpha^k d^k$ mit $$ \alpha ^k = \frac{(r^k)^Td^k}{(d^k)^TAd^k}$$
$r^{k+1} = b- Au^{k+1}$ (äquivalent $r^{k+1} = r^k- \alpha^k Ad^{k}$)\
$ d^{k+1} = r^{k+1} + \beta^{k}d^k$ mit $$ \beta^k = - \frac{(r^{k+1})^T Ad^k}{(d^k)^TAd^k}$$ 


Mit Preconditioning:

1. Startwerte: Startpunkt $u^0$, Residuum, $r^0 = b- Au^0$, $z^0 = C^{-1}r^0$, Suchrichtung $d^0 = z^0$
2. wiederhole für $k = 0,.., \min \{ $ gewünschte Genauigkeit erreicht, $ n  \}$: \
$ u^{k+1} = u^k + \alpha^k d^k$ mit $$ \alpha ^k = \frac{(z^k)^Tr^k}{(d^k)^TAd^k}$$
$r^{k+1} = b- Au^{k+1}$  (äquivalent $r^{k+1} = r^k- \alpha^k Ad^{k}$)\
$z^{k+1} = C^{-1}r^{k+1}$
$ d^{k+1} = z^{k+1} + \beta^{k}d^k$ mit $$ \beta^k = \frac{(z^{k+1})^T r^{k+1}}{(z^k)^T r^k}$$ 

<!-- Falls Zeit über das $\beta$ nachdenken. -->


#### Additive Schwarz

Angenommen $u \in \R^n$ kann für $E_i \in \R^{n \times n_i}$ mit Rang $n_i$ wie folgt geschrieben werden
$$
    u = \sum_i E_i u_i \tag{*}
$$

dann definiere

$$
C^{-1} = \sum_i E_i A_i^{-1}E_i^T \quad \text{mit} \quad A_i = E_i^T A E_i.
$$


Es gilt

$$ 
u^T C u = \inf_{u \text{ erfüllt } (*) } \sum_i u_i^T A_i u_i.
$$

#### Äquivalenz Summennorm und H(Curl)-Norm

Aufteilung von für $ u \in H(\Curl)$ in

$$
u  = \nabla \phi + q \quad \text{ mit } \phi \in H^1 , ~ q \in (H^1)^3
$$


Summenraumnorm im nicht gewichten Fall

$$
||u||_s^2 := \inf_{v = \nabla \phi + q } || \phi ||^2_{H^1} + || q ||^2_{H^1} 
$$ 


Zu zeigen Normen sind equivalent dh: $||u||_s^2 \lesssim ||u||_{\Curl} \lesssim ||u||_s $.

Richtung $||u||_{\Curl} \lesssim ||u||_s $: Dreiecksungleichung  + Young-Ungleichung (wenn ich mich nicht verrechnet habe).

Richtung $||u||_s \lesssim ||u||_{\Curl}$: In http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.581.5885&rep=rep1&type=pdf (Lemma 2.2) werden für $u$ mit $\Div u = 0$ (Noch herausfinden: Wo wird das im Beweis benötigt? Ist das eine Einschränkung?) folgende Abschätzungen gezeigt (ohne Quadrate aber dann stimmt das auch):

$$
|| q ||_{L^2}^2 + || \phi ||_{H^1}^2 \lesssim || u ||_{L^2}^2 \quad \text{ und } \quad  || \Grad q ||_{L^2}^2 \lesssim || \Curl u ||_{L^2}^2
$$


Gewichtete $H(\Curl)$-Norm:

<!-- $$
\langle \langle u,v \rangle \rangle :=  \mu \langle \Curl u,\Curl v \rangle_{L^2} + \kappa \langle  u, v \rangle_{L^2}
$$ -->

$$
|||u|||_{\Curl}^2 :=  \mu ||\Curl u ||_{L^2}^2 + \kappa || u ||_{L^2}^2
$$

Damit ergibt sich die Abschätzung 

$$
\kappa (|| q ||_{L^2}^2 + || \phi ||_{H^1}^2) + \mu || \Grad q ||^2_{L^2 }\lesssim |||u|||_{\Curl}^2
$$

Analog für Skalarprodukte. Passt somit zum ams.py Beispiel.

<!-- Summenraumnorm:
$$
||u||_s^2 := \inf_{v = \nabla \phi + q } | \mu ||\Curl v ||_{L^2}^2
$$ -->

